# Prompt Engineering: Use OpenAI to Analyze Twitter Data 
This is a simple tutorial teaching prompt engineering basics and analyzing Twitter data with OpenAI large language models (LLM).
Please purchase an [OpenAI API](https://openai.com/index/openai-api/) and store it in a safe place. This tutorial uses [AWS Secretes Manager](https://aws.amazon.com/secrets-manager/) to store the API keys.  

## Large Language Model Basics
LLM repeatable predicts the next world using supervised learning. To predict the following sentence: 

`Learning data science in the cloud with AI`

A model needs to learn to predict the following steps:

|Input|Output|
|:---|---|
|Learning data science |in |
|Learning data science in |the | 
|Learning data science in the |cloud |
|Learning data science in the cloud |with |
|Learning data science in the cloud with |AI|

To train an LLM model:
1. Training a base LLM model on a large amount of training data to predict the next word 
2. Fine-tune on examples where outputs follow instructions in the input 
3. Human rates quality of different LLM outputs 
4. Tune LLM to generate outputs with higher rates using RLHF (Reinforcement learning from human feedback)

## Set up OpenAI Models

Load the API keys with AWS Secrets Manage Function 

In [1]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Install Python libraries.

- pymongo: manage the MongoDB database
- openai: call the OpenAI APIs.

In [2]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 18.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Load the OpenAI API key and define a `openai_help` function.

In [4]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

Temperature: 
- Low temperature: always choose the most likely response, reliable, predictable responses  
- High temperature: diverse responses, more creative responses

Tokens and Models: 
- LLM predicts tokens, which are commonly occurring sequences of characters. 
- One token is about four characters in English, and 100 tokens are roughly 75 words. Check [token estimate](https://platform.openai.com/tokenizer).
- Different models can process various amounts of tokens at different performance levels and costs. Check [OpenAI models](https://platform.openai.com/docs/models) for more details.

Roles:
- system: specify the overall tone or behavior of the assistant 
- user: instruction given to the LLM
- assistant: LLM responded content, we also can provide content in few-shot promoting or histories of conversations


A simple example using [gtp-4o](https://platform.openai.com/docs/models/gpt-4o) and temperature 0.

In [5]:
messages = [{"role": "user", "content": "What is the capital of USA"}]

print(openai_help(messages))

The capital of the United States is Washington, D.C.


Add a system message asking LLM to act as a high school teacher with different temperatures.

In [8]:
messages = [
    {"role": "system", "content": "use tone as a high school teacher"},
    {"role": "user", "content": "What is the capital of USA"}
    ]

print(openai_help(messages, temperature = 0.8))

The capital of the United States is Washington, D.C. It's an important city politically and historically. If you have any other questions about the U.S. or related topics, feel free to ask!


Add assistant messages to teach LLM what `##` is.

In [10]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

It is 33.


## Prompt Engineering Principles 
- Use delimiters to separate different parts of a prompt to provide clear instructions and prevent prompt injections.
- Structure outputs in JSON documents or other formats to use the outputs in subsequent steps 
- Few-shot promoting: provide successful examples of a task and then ask the model to perform a similar task. 
- Chain of thought reasoning: request a series of reasoning steps in prompts to help the model achieve correct answers
- Chain of prompts: split a task into multiple prompts where each prompt can focus on a sub-task at a time and take different actions at different stages. It saves tokens, is easier to test, can involve human input, or use external tools.
- Interactive process 
  1. Try something first 
  2. Analyses the result, identify errors, and redefine the prompt 
  3. Test the prompts with different datasets 


An example using delimiters, structured output and few-shot promoting:

In [11]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

{ "sentiment": "positive" }


## Analyze Twitter data

### Connect to the MongoDB cluster

In [12]:
import pymongo
from pymongo import MongoClient
mongodb_connect = get_secret('mongodb')['connection_string']

mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'tweet.id_1'

### Extract Tweets

In [13]:
filter={

    
}
project={
    'tweet.text': 1, 
    'tweet.id': 1
}
#rename the client to mongo_client
result = mongo_client['demo']['tweet_collection'].find(
  filter=filter,
  projection=project
)

In [14]:
tweet_data = []
for tweet in result:
    tweet_data.append(tweet['tweet']['text'])

In [15]:
print('Number of tweets: ',len(tweet_data))

Number of tweets:  97


### Summarization 
- Analyze election tweets with delimiters 
- Change the size of the summarization 
- Summarize tweets and focus on different perspectives. 

In [16]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets primarily focus on the Premier League, with discussions about players like Bruno Fernandes and Wayne Rooney, and predictions for upcoming matches. Bruno Fernandes is highlighted as a potential all-time great midfielder, and he has been named Premier League Player of the Month for March 2025. There are also mentions of prize giveaways related to Premier League matches, and discussions about team performances, including Arsenal and Manchester United. Additionally, there are references to other leagues and competitions, such as the FA Cup and the potential for CPL soccer in Quebec. Some tweets also touch on financial aspects of clubs like Chelsea and the impact of player sales.


In [17]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

Bruno Fernandes is highlighted as a top Premier League midfielder, winning Player of the Month. Premier League action resumes.


In [19]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    focus on how people discuss Bruno Fernandes,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

Bruno Fernandes is frequently discussed as a potential all-time great Premier League midfielder. He recently won the Premier League Player of the Month for March 2025, highlighting his significant impact. With 62 goals, only 10 midfielders have scored more in Premier League history, underscoring his impressive performance.


### Moderation 
- Iterate each tweet and use the [moeration endpoint](https://platform.openai.com/docs/api-reference/moderations) to identify flagged tweets
- Print flagged tweets


In [20]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)
                print(tweet)

In [21]:
for tweet in tweet_data:
    flag_help(tweet)

===
sexual
RT @GingeUpdates: 🚨| AngryGinge and John Terry bond over the Arsenal Baller League and Arsenal Premier League team being bottlejobs 😭 https…
===
sexual
RT @GingeUpdates: 🚨| AngryGinge and John Terry bond over the Arsenal Baller League and Arsenal Premier League team being bottlejobs 😭 https…
===
sexual
RT @GingeUpdates: 🚨| AngryGinge and John Terry bond over the Arsenal Baller League and Arsenal Premier League team being bottlejobs 😭 https…
===
sexual
RT @GingeUpdates: 🚨| AngryGinge and John Terry bond over the Arsenal Baller League and Arsenal Premier League team being bottlejobs 😭 https…
===
sexual
RT @GingeUpdates: 🚨| AngryGinge and John Terry bond over the Arsenal Baller League and Arsenal Premier League team being bottlejobs 😭 https…


### Transforming
- Translating to a different language 
- Transform tones, such as formal vs. informal.  


In [22]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into Chinese"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

转发 @premierleague：布鲁诺·费尔南德斯正在成为英超历史最佳中场球员之一吗？

➡️ https://t.co/r371zbtua4 https://t.co/hiiqnRDdM1
转发 @Xyes_com: 60美元USDT奖品！💥（2名获奖者，每人30美元）✨  

#诺丁汉森林 vs #曼联 – 英超对决 ⚽️  

如何参与：
🔄…
RT @northtribune: 我们能在魁北克举办@CPLsoccer 吗？

联盟将在本周三于拉瓦尔大学发布公告…
RT @cory__1_: £200变成£27,282💷😍

英超联赛卡片累积投注🟨🟨 ⚽️

如果赢了，我会给❤️并转发这条推文的两个人每人1000。

B…
RT @livescore: 队长太棒了 🤩

@B_Fernandes8 获得了我们2025年3月的英超月度最佳球员奖 🔴🏆 https://t.co/XvPuPh7VEC
RT @G_Yakovleff: “关于意大利 🇮🇹 目前是个否定。如果我们准备与几个国家在乌克兰 🇺🇦 进行一项大规模行动，而意大利…
RT @livescore: 队长太棒了 🤩

@B_Fernandes8 赢得了我们2025年3月的英超月度最佳球员奖 🔴🏆 https://t.co/XvPuPh7VEC
⚔️在一个没有英超联赛的周末之后，本周二，城市球场将迎来第30轮比赛，由两位葡萄牙教练展开对决。

✍🏻作者：@mikelbrito11 
💻详情请见：

https://t.co/2SxJJgbdih
RT @Utdjacobi_: 韦恩·鲁尼被普通球迷过于不尊重，我们谈论的是英超历史上最好的前锋。htt…
@GoMavenHub 我认为可能是英超
@GoMavenHub 我认为英超
RT @livescore: 队长太棒了 🤩

@B_Fernandes8 荣获我们2025年3月的英超月度最佳球员 🔴🏆 https://t.co/XvPuPh7VEC
RT @AdamJoseph____: 在英超历史上，只有10名中场球员的进球数超过布鲁诺·费尔南德斯（62个）。他的进球数与……
英超联赛回归 4月1日和2日 https://t.co/vAnnntzpbP
RT @Footballogue: 看到这些曼彻斯特城球员在英超29轮比赛后数据如此低迷，真是太

KeyboardInterrupt: 

In [23]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Yoda """},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

Becoming an all-time Premier League midfielder, is Bruno Fernandes? Hmm.
RT @Xyes_com: Prize of $60 USDT, there is! 💥 (2 winners, $30 each, hmm) ✨  

#NottinghamForest vs #ManUnited – Premier League Showdown, it is ⚽️  

Enter, how you must: 🔄…
RT @northtribune: In Quebec, could we get @CPLsoccer taking place, hmm? 

An announcement at Laval University this Wednesday, the league will be making…
£200 INTO £27,282, hmmm? Premier League card acca, it is. If win, this does, 1000 to 2 people who ❤️ and retweet, I shall give. Yes, hmmm.
Captain fantastic, hmm? 🤩 Our Premier League Player of the Month for March 2025, @B_Fernandes8 wins, yes. 🔴🏆 https://t.co/XvPuPh7VEC
RT @G_Yakovleff: "A NO, it is, regarding Italy 🇮🇹 right now. Prepare for a big operation in Ukraine 🇺🇦 with several countries, if we do, and Italy...
Captain fantastic, hmm? 🤩 Our Premier League Player of the Month for March 2025, @B_Fernandes8 is. 🔴🏆 Won, he has. https://t.co/XvPuPh7VEC
⚔️Without Premier League, a weekend has p

KeyboardInterrupt: 

### Inferring
- Use step-by-step instructions with delimiters to:
  1. Identify sentiments
  2. Identify emotions
  3. Extract mentioned people's names
  3. Identify whether a tweet supports Democratic, Republican, or unknown 
  4. Extract outputs into a structured JSON document. 
- Identify topics from Tweets. 


In [24]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a single word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    print(openai_help(messages))

{
  "sentiment": "neutral",
  "emotion": "curiosity",
  "mentioned": ["Bruno Fernandes"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "excitement",
  "mentioned": ["Xyes_com"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "anticipation",
  "mentioned": ["@northtribune", "@CPLsoccer"],
  "support": "neutral"
}
{
  "sentiment": "positive",
  "emotion": "excitement",
  "mentioned": ["cory__1_"],
  "support": "neutral"
}
{
  "sentiment": "positive",
  "emotion": "excitement",
  "mentioned": [
    "B_Fernandes8"
  ],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "uncertain",
  "mentioned": ["G_Yakovleff"],
  "support": "neutral"
}
{
  "sentiment": "positive",
  "emotion": "excitement",
  "mentioned": ["@livescore", "@B_Fernandes8"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "anticipation",
  "mentioned": ["@mikelbrito11"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "respect",
  "men

KeyboardInterrupt: 

In [25]:

messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

{
  "topics": [
    "Bruno Fernandes' performance in the Premier League",
    "Premier League match predictions and betting",
    "Premier League Player of the Month awards",
    "Premier League fixtures and schedules",
    "Wayne Rooney's legacy in the Premier League",
    "Premier League team performances and standings",
    "Premier League ticket sales and availability",
    "Premier League club financials and investments",
    "Comparisons of Premier League players and teams",
    "Premier League fan engagement and social media interactions"
  ]
}


### Expanding with multiple prompts 
- Identify which party receives majority supports
- Provide contexts in the system message
- Create a chatbot to answer users’ inquiry  


In [26]:
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_data):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a singple word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


100%|██████████| 97/97 [01:41<00:00,  1.05s/it]


In [27]:
print(analysis_result)

['{\n  "sentiment": "neutral",\n  "emotion": "curiosity",\n  "mentioned": ["Bruno Fernandes"],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "excitement",\n  "mentioned": ["Xyes_com"],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "anticipation",\n  "mentioned": ["@northtribune", "@CPLsoccer"],\n  "support": "neutral"\n}', '{\n  "sentiment": "positive",\n  "emotion": "excitement",\n  "mentioned": ["cory__1_"],\n  "support": "neutral"\n}', '{\n  "sentiment": "positive",\n  "emotion": "admiration",\n  "mentioned": ["@B_Fernandes8"],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "uncertain",\n  "mentioned": ["G_Yakovleff"],\n  "support": "neutral"\n}', '{\n  "sentiment": "positive",\n  "emotion": "excitement",\n  "mentioned": ["@B_Fernandes8"],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "anticipation",\n  "mentioned": ["@mikelbrito11"],\n  "support": "neutral"\n}', '{\n  "sentime

In [28]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis reuslt delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets that support Democratic and Republican;
                                        step 2 {delimiter} identify the common sentiments and emotoions to each mentioned people;
                                        step 3 {delimiter} organize the result in a json document with keys <Democratic count>, <Republican count>, <people name>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

{
  "Democratic count": 0,
  "Republican count": 0,
  "people name": {
    "Bruno Fernandes": {
      "sentiments": ["neutral", "positive"],
      "emotions": ["curiosity", "admiration", "informative"]
    },
    "Xyes_com": {
      "sentiments": ["neutral"],
      "emotions": ["excitement"]
    },
    "@northtribune": {
      "sentiments": ["neutral"],
      "emotions": ["anticipation"]
    },
    "@CPLsoccer": {
      "sentiments": ["neutral"],
      "emotions": ["anticipation"]
    },
    "cory__1_": {
      "sentiments": ["positive"],
      "emotions": ["excitement"]
    },
    "@B_Fernandes8": {
      "sentiments": ["positive"],
      "emotions": ["admiration", "excitement"]
    },
    "G_Yakovleff": {
      "sentiments": ["neutral"],
      "emotions": ["uncertain"]
    },
    "@mikelbrito11": {
      "sentiments": ["neutral"],
      "emotions": ["anticipation"]
    },
    "Wayne Rooney": {
      "sentiments": ["neutral"],
      "emotions": ["respect", "admiration"]
    },
    "Go

## Create a chatbot

In [29]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are a chabot answer user questions based on the tweets,
                                {delimiter}{tweet_data}{delimiter}, 
                                if user mentioned a people name in the {delimiter}{analysis_summary}{delimiter} people field,report the corresponding sentiment and emotion,
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    print(f"Chatbot: {reply}")

You:  what do they talk about?


Chatbot: In the provided tweets, people are mainly discussing topics related to football, specifically the Premier League. Key points of discussion include:

1. **Bruno Fernandes**: His potential to become an all-time great Premier League midfielder is questioned. There's admiration for his achievements and a curiosity about his place among top midfielders.

2. **Football Matches and Predictions**: There are mentions of upcoming fixtures like Nottingham Forest vs. Man United and predictions for Premier League Matchday 30 outcomes. Chris Sutton's predictions and their implications on the league table are discussed with anticipation.

3. **Players' Performances and Transfers**: Discussion about players like Viktor Gyokeres, Alexander Isak, and their performance in different leagues. Wayne Rooney is highlighted as being a top striker, while Cristiano Ronaldo's ranking as the Premier League's greatest forward is debated.

4. **League and Team Updates**: News related to various football clu

You:  who is bruno fernandes


Chatbot: Bruno Fernandes is a professional footballer from Portugal who plays as a midfielder. He is well-known for his time in the Premier League, playing for Manchester United. Fernandes is recognized for his exceptional skills in playmaking, goal scoring, and his ability to influence the game with his vision and passing. Over his career at Manchester United, he has become one of the standout players in the team, earning accolades and contributing significantly to the team's performances. His impact in the Premier League is often discussed, with many considering him to be on a path to becoming one of the all-time great midfielders in the league.


You:  how good are arsenal?


Chatbot: Arsenal is considered one of the top football clubs in the English Premier League, known for its rich history and competitive performances. The club has a reputation for playing attractive, attacking football and has been home to numerous legendary players and managers. 

In recent seasons, Arsenal has seen a resurgence under the management of Mikel Arteta. The team has shown strong performances in the league and has been a contender for top positions, including qualification for European competitions like the UEFA Champions League. Individual players like Bukayo Saka have received praise for their contributions to the team's success, showcasing significant talent and potential.

Overall, Arsenal is seen as a competitive and formidable team in the Premier League landscape, with a blend of experienced players and rising stars, ambitious to achieve domestic and European success.


You:  Who is the best player in premier league history


Chatbot: Determining the "best" player in Premier League history is subjective and often depends on personal preferences and criteria such as individual skill, impact on the game, and achievements. However, several players are frequently mentioned in discussions about the greatest in Premier League history:

1. **Thierry Henry**: The French striker was instrumental for Arsenal, known for his incredible goal-scoring ability and flair. He won multiple Premier League titles and was renowned for his technical skill and consistency.

2. **Cristiano Ronaldo**: Before becoming a global superstar at Real Madrid, Ronaldo made a significant impact at Manchester United, winning multiple league titles and individual awards, showcasing his immense talent.

3. **Ryan Giggs**: A Manchester United legend, Giggs had a remarkable career, winning numerous league titles and being known for his longevity, versatility, and class on the pitch.

4. **Alan Shearer**: The Premier League's all-time top scorer, S

You:  How good is jack grealish


Chatbot: Jack Grealish is a professional footballer known for his creativity, dribbling skills, and flair on the field. As of recent years, he has played in the Premier League for Manchester City after transferring from Aston Villa, where he was the captain and a standout performer.

Grealish is highly regarded for his ability to control the game and create scoring opportunities for his teammates. His dribbling prowess and ability to draw fouls have made him a valuable asset to any team. At Manchester City, he continues to develop under the management of Pep Guardiola, adapting to a team known for its attacking prowess and tactical discipline.

While there were initial periods of adaptation at Manchester City, his performances have shown growth and increased influence in important games. Grealish's talent and potential continue to make him one of the exciting players to watch in the Premier League, offering a unique combination of skill, agility, and vision on the field.


## Reference
- Isa Fulford and Andrew Ng. n.d.-a. *“Building Systems with the ChatGPT API.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/.
- ———. n.d.-b. *“ChatGPT Prompt Engineering for Developers.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/.
- OpenAI. n.d. *“OpenAI Documents.”* OpenAI. Accessed October 18, 2024. https://platform.openai.com.
